# Plotting HEALPix Data with XDGGS and Lonboard

## Authors & Contributors
### Authors
- Tina Odaka, Ifremer (France), [@tinaok](https://github.com/tinaok)

## Purpose
In this notebook, we will demonstrate several examples of plotting HEALPix data.  
This notebook is prepared for a poster presentation at the 3rd Destination Earth User eXchange, held from October 15 - October 16, 2024, in Darmstadt. You can find more information about the event here: [3rd Destination Earth User eXchange](https://destination-earth.eu/event/3rd-destination-earth-user-exchange/).

Link to the poster on Zenodo: [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.13934967.svg)](https://doi.org/10.5281/zenodo.13934967)

## Setup

To run this notebook, you need to install XDGGS along with Lonboard. You can find the repository here: [XDGGS GitHub Repository](https://github.com/xarray-contrib/xdggs.git).

To install the necessary dependencies, use the following command:

```bash
pip install "xdggs[explore] @ git+https://github.com/xarray-contrib/xdggs.git"```

This command installs the XDGGS package directly from its GitHub repository. The `@` symbol indicates that the package is being installed from a specific URL, in this case, GitHub. The `explore` extra is specified to ensure that additional dependencies required for exploring and visualizing data (like `lonboard`) are included in the installation.

This will install all the required libraries, such as:

- `xarray`
- `numpy`
- `xdggs`
- `lonboard`
- `arro3-core`
- `matplotlib`

## Why Use Lonboard for Plotting in XDGGS?

Lonboard uses GPU-based rendering, specifically leveraging WebGL through deck.gl. This enables efficient handling of large datasets, allowing for fast, interactive visualization of geospatial data directly in a browser. GPU-based rendering is essential when working with large-scale datasets that could slow down or crash traditional plotting libraries.

Lonboard's high performance stems from its use of binary data formats such as GeoArrow and GeoParquet, which minimize data transfer and processing overhead. Unlike traditional libraries that rely on text formats like GeoJSON, Lonboard maintains a binary format throughout the entire pipeline. This approach reduces the number of data copies and avoids costly format conversions, leading to faster data transfers and more efficient rendering.

In [ ]:
import warnings

import cf_xarray.units  # noqa: F401
import numpy as np
import pint_xarray  # noqa: F401
import xarray as xr

import xdggs

warnings.filterwarnings("ignore")

##  Plotting generated test dataset
HEALPix is designed to represent the sphere using spherical harmonics functions.
Following code with generate a test dateset. 

In [ ]:
import healpy as hp
import matplotlib.pyplot as plt

lmax = 3
level = 8
nside = 2**level
# compute Alm
# get the l and m availble for l<=lmax
l, m = hp.Alm.getlm(lmax=lmax)  # noqa: E741

# count the number of alm map (1 for m=0 and 2 for m>0)
n_alm = (m == 0).sum() + 2 * (m > 0).sum()
function = np.zeros([n_alm, 12 * nside**2])

alm = np.zeros([l.shape[0]], dtype="complex")

i = 0

# array to store the l and m values of the A_lm
l_func = np.zeros(n_alm, dtype="int")
m_func = np.zeros(n_alm, dtype="int")
is_real_func = np.zeros(n_alm, dtype="int")

for k in range(l.shape[0]):
    alm[k] = 1.0
    function[i] = hp.reorder(hp.alm2map(alm, nside), r2n=True)
    l_func[i] = l[k]
    m_func[i] = m[k]
    is_real_func[i] = 1
    i += 1
    if m[k] > 0:
        alm[k] = complex(0, 1)
        function[i] = hp.reorder(hp.alm2map(alm, nside), r2n=True)
        l_func[i] = l[k]
        m_func[i] = m[k]
        is_real_func[i] = 0
        i += 1
    alm[k] = 0.0
lm = 3
plt.figure(figsize=(12, 5))
for k in range(l_func.shape[0]):
    pos = (
        1
        + l_func[k] * (2 * lm + 1)
        + 2 * (is_real_func[k] - 0.5) * m_func[k]
        - 1
        + (lm + 1)
    )
    if is_real_func[k] == 1:
        title = "$\mathbb{R}(A_{\ell=%d,m=%d})$" % (l_func[k], m_func[k])  # noqa: W605
    else:
        title = "$\mathbb{I}(A_{\ell=%d,m=%d})$" % (l_func[k], m_func[k])  # noqa: W605
    if l_func[k] <= lm:
        hp.mollview(
            function[k],
            nest=True,
            hold=False,
            sub=(lm + 1, 2 * lm + 1, pos),
            title=title,
            cbar=False,
            cmap="coolwarm",
        )

In [ ]:
level = 8
cell_ids = np.arange(12 * 4**level)
grid_info = {"grid_name": "healpix", "level": level, "indexing_scheme": "nested"}

ds = (
    xr.Dataset(coords={"cell_ids": ("cells", cell_ids, grid_info)})
    .pipe(xdggs.decode)
    .pipe(lambda ds: ds.merge(ds.dggs.cell_centers()))
    .assign(
        data=lambda ds: np.cos(6 * np.radians(ds["latitude"]))
        * np.sin(6 * np.radians(ds["longitude"]))
        # data=function[k,:]
    )
)
SH_L3_M2 = xr.DataArray(
    function[13, :], dims=("cells"), coords={"cell_ids": ("cells", cell_ids, grid_info)}
)
ds["SH_L3_M2"] = SH_L3_M2
ds

### Plotting with xdggs is very simple, just use 'explore' function

In [ ]:
ds["SH_L3_M2"].dggs.explore(center=0, cmap="coolwarm", alpha=0.8)

## Open a sample DestinE data in hearlpix (saved in zarr format with attributes)

It is straigh forward to open dataset in xarray to use an annotated zarr file.  

Here we have loaded grib file then transformed it to zarr file with following metadata, and assigned coordinate.  

```bash
.assign_coords(
    cell_ids=('value', np.arange(12*4**10 ), 
              {"grid_name":"healpix","level": 10, "indexing_scheme": "nested"}))
```

In [ ]:
ds = xr.open_dataset(
    # "https://data-taos.ifremer.fr/DestinE/average_surface_temperature.zarr",
    "DestinE/average_surface_temperature.zarr",
    engine="zarr",
    chunks={},
    consolidated=True,
)
ds

In [ ]:
data = (
    ds["avg_thetao"]
    .isel(oceanModelLayer=1)
    .pint.quantify()
    .pint.to({"avg_thetao": "degC"})
    .pint.dequantify()
    .pipe(xdggs.decode)
    .compute()
)
data

In [ ]:
hp.cartview(data.data, nest=True, cmap="viridis", flip="geo")

In [ ]:
data.dggs.explore(center=0, cmap="viridis", alpha=0.5)

### With XDGGS, one can select points of data not only from cell_ids from DGGS, but also from latitude and longitude.

In [ ]:
lon = np.arange(48, 49.5, 0.01)
lat = np.arange(354, 355, 0.01)
full_lat = np.repeat(lat, len(lon))
full_lon = np.tile(lon, len(lat))

(data.dggs.sel_latlon(longitude=full_lon, latitude=full_lat)).dggs.explore(
    center=0, cmap="viridis", alpha=0.8
)

## Open a EERIE Cloud  data in hearlpix, saved in zarr format from DKRZ.

NextGems project provide their data set in healpix in zarr format. 
Deatiled description of EERIE Cloud can be found here
https://easy.gems.dkrz.de/simulations/EERIE/eerie_data-access_online.html

After opening we need to assigne coordinate and meta data defining the grid system.    



In [ ]:
ds = xr.open_dataset(
    "https://eerie.cloud.dkrz.de/datasets/nextgems.IFS_9-FESOM_5-production.2D_hourly_healpix512/zarr",
    engine="zarr",
    chunks={},
    consolidated=True,
)
ds

In [ ]:
ds = ds.assign_coords(
    cell_ids=(
        "value",
        np.arange(12 * 4**9),
        {"grid_name": "healpix", "level": 9, "indexing_scheme": "nested"},
    )
)

In [ ]:
data = ds["msl"].isel(time=1000).compute().pipe(xdggs.decode)
data

In [ ]:
data.dggs.explore(center=0, cmap="viridis", alpha=0.8)

### Selecting region to plot
#### Selection using healpix cell_ids

In [ ]:
data.sel(cell_ids=slice(0, 1000)).dggs.explore(cmap="viridis", alpha=0.8)  # center=0,

#### Selection using latitude and longitude

In [ ]:
(
    data.dggs.sel_latlon(latitude=0, longitude=48)
    .expand_dims("cell_ids")
    .pipe(xdggs.decode)
).dggs.explore(center=0, cmap="viridis", alpha=0.8)

In [ ]:
(data.dggs.sel_latlon(longitude=[48, 47, 46], latitude=[-5, -5.3, -5.5])).dggs.explore(
    center=0, cmap="viridis", alpha=0.8
)

In [ ]:
lon = np.arange(48.5, 49, 0.01)
lat = np.arange(355, 355.5, 0.01)
full_lat = np.repeat(lat, len(lon))
full_lon = np.tile(lon, len(lat))

(data.dggs.sel_latlon(longitude=full_lon, latitude=full_lat)).dggs.explore(
    center=0, cmap="viridis", alpha=0.8
)

## Current Issue with Testing Different Datasets

It is crucial to define a consistent coordinate 'name' and establish how attributes should be specified in order to test different data sources without having to re-define the coordinate name for each dataset. In **XDGGS**, we follow the convention below:

```python
.assign_coords(
    cell_ids=('value', np.arange(12*4**10), 
              {"grid_name": "healpix", "level": 10, "indexing_scheme": "nested"}))
```

## Whats next

Currently, selecting data by latitude and longitude must be done on a point-by-point basis. However, the conventional approach for users to select a region of interest is still a bounding box (bbox) using latitude and longitude. Therefore, enabling this functionality is a priority to streamline data selection.